Please provide your names, last names, and student IDs.

* name 1, last name 1, student id 1
* name 2, last name 2, student id 2 (when working in a pair)

# DECISION AIDING

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import common as cm
import matplotlib.pyplot as plt

This exercise consists of 3 parts. Finish the first part to get a mark of 3.0. The first two parts to get 4.0. Finish all exercies to get 5.0.

# Part 1: Concordance

Given are the following modes of transport: bus, bike, car, train. Each mode is characterized by 2 cost-type criteria: price and time; and 2 gain-type criteria: comfort and reliability. 

Mode of transport | Time | Comfort | Price | Reliability
--- | --- | --- | --- | ---
 **bus**  | 6 | 3  | 6 | 2
 **bike** | 8 | 2  | 2 | 8
 **car**  | 2 | 10 | 9 | 7
 **train**| 3 | 5  | 5 | 6
 **by foot**| 10 | 2  | 0 | 5

In [4]:
data = {
    'mode': ['bus', 'bike', 'car', 'train', 'foot'],
    'time': [6, 8, 2, 3, 10],
    'comfort': [3, 2, 10, 5, 2],
    'price': [6, 2, 9, 5, 0],
    'reliability': [2, 8, 7, 6, 5]
}
criteria = ['time', 'comfort', 'price', 'reliability']

data = pd.DataFrame(data, columns=['mode', 'time', 'comfort', 'price', 'reliability'])
data

,mode,time,comfort,price,reliability
0,bus,6,3,6,2
1,bike,8,2,2,8
2,car,2,10,9,7
3,train,3,5,5,6
4,foot,10,2,0,5


1.2) Below are the parameters, i.e., threholds, criterion-type, and weights, for each criterion, 

In [5]:
parameters = {
 'time': {'weight': 4, 'q': 1.0, 'p': 2, 'v': 4, 'type': 'cost'},
 'comfort': {'weight': 2, 'q': 2.0, 'p': 3, 'v': 6, 'type': 'gain'},
 'price': {'weight': 3, 'q': 1.0, 'p': 3, 'v': 5, 'type': 'cost'},
 'reliability': {'weight': 1, 'q': 1.5, 'p': 3, 'v': 5, 'type': 'gain'}
    }

sum_weights = 10.

pd.DataFrame(parameters, columns=['time', 'comfort', 'price','reliability']).reindex(['type', 'q', 'p', 'v', 'weight']).T

,type,q,p,v,weight
time,cost,1,2,4,4
comfort,gain,2,3,6,2
price,cost,1,3,5,3
reliability,gain,1.5,3,5,1


1.3) Finish the below function for calculating a marginal concordance for $i$-th criterion (gain type) $c_i(g_i(A),g_i(B))$ based on q and p tresholds. 

In [6]:
def getConcordanceCost(gA, gB, q, p):
    if gB - gA >= -q:
        return 1
    if gB - gA <= -p:
        return 0
    else:
        return (p - (gA - gB)) / (p - q)

def getConcordanceGain(gA, gB, q, p):
    if gA - gB >= -q:
        return 1
    if gA - gB <= -p:
        return 0
    else:
        return (p - (gB - gA)) / (p - q)
print(getConcordanceGain(700, 750, 25, 125))
print(getConcordanceCost(225, 170, 10, 25))
# print(pd.DataFrame(data, columns=['mode', 'time', 'comfort', 'price', 'reliability']))
# print(pd.DataFrame(parameters, columns=['time', 'comfort', 'price','reliability']).reindex(['type', 'q', 'p', 'v', 'weight']).T)

0.75
0


1.4)  Calculate  comprehensive concordance  index  for  all  criteria  of  alternatives A and B. Remember that comprehensive concordance must be divided by the sum of weights (normalization).  

In [7]:
def getComprehensiveConcordance(A, B, criteria, parameters):
    fst_idx = 0
    scd_idx = 0
    if A == B:
        return 0
    for i in range(len(data['mode'])):
        if A == data['mode'][i]:
            fst_idx = i
        if B == data['mode'][i]:
            scd_idx = i
    sus = zip(data['time'], data['comfort'], data['price'], data['reliability'])
    appendix = []
    iterator = 0
    bob = []
    for f in sus:
        appendix.append(f)
    for j in criteria:
        if parameters[j]['type'] == 'cost':
            bob.append(getConcordanceCost(appendix[fst_idx][iterator],appendix[scd_idx][iterator],parameters[j]['q'],parameters[j]['q']))
        if parameters[j]['type'] == 'gain':
            bob.append(getConcordanceGain(appendix[fst_idx][iterator],appendix[scd_idx][iterator],parameters[j]['q'],parameters[j]['q']))
        iterator += 1
    iterator = 0
    for j in criteria:
        bob[iterator] = bob[iterator] * parameters[j]['weight']
        iterator += 1
    concordance = sum(bob)
    return concordance / sum_weights

1.5) Check comprehensive concordance indexes for C(bus, some transportation) (HINT: for C(0,1) should be 0.6):

In [8]:
for alternative_id, alternative_row in data.iterrows():
    print("C({0},{1}) = ".format(0, alternative_id), getComprehensiveConcordance(data.loc[0][0], alternative_row[0], criteria, parameters))

C(0,0) =  0
C(0,1) =  0.6
C(0,2) =  0.3
C(0,3) =  0.5
C(0,4) =  0.6


1.6) Finish the below function for generating a concordance matrix. Use a majority_threshold as a cutting-level. For hich pairs a concordance is fulfilled?

In [275]:
def getConcordanceMatrix(data, criteria, parameters, majority_treshold=0.7):
    concordance_matrix = np.zeros((len(data),len(data)))
    iterator = 0
    for f in data['mode']:
        bomba = 0
        for s in data['mode']:
            vall = getComprehensiveConcordance(f, s, criteria, parameters)
            if vall >= 0.7:
                vall = 1
            else:
                vall = 0
            concordance_matrix[iterator][bomba] = vall
            bomba += 1
        iterator += 1
    return concordance_matrix
            

In [276]:
print(getConcordanceMatrix(data, criteria, parameters))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 1. 0. 1. 1.]
 [1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0.]]


HINT :): The resulting matrix should be: <br><br>[[0. 0. 0. 0. 0.]<br>
 [0. 0. 0. 0. 1.]<br>
 [1. 1. 0. 1. 1.]<br>
 [1. 0. 1. 0. 1.]<br>
 [0. 0. 0. 0. 0.]]<br>

# Part 2: outranking graph

2.1) Complete the function for calculating a marginal discordance  $d_i(𝐴,𝐵)$ based on v treshold. $d_i(𝐴,𝐵) = 1$ when A is worse than B on criterion $i$ by at least the veto threshold, zero otherwise.

In [ ]:
def getDiscordanceGain(A, B, v):
    #TODO
    return 0

def getDiscordanceCost(A, B, v):
    return getDiscordanceGain(B, A, v)


2.2) Calculate a comprehensive discordance index. $D(a,b) = 1$ if at least one criterion vetoes against aSb.

In [ ]:
def getComprehensiveDiscordance(A, B, criteria, parameters): 
    #TODO
    return 0.

2.3) Check comprehensive discordance indexes for D(bus, some transportation):

In [ ]:
for alternative_id, alternative_row in data.iterrows():
    print("D({0},{1}) = ".format(0, alternative_id),getComprehensiveDiscordance(data.loc[0], alternative_row, criteria, parameters))


2.4) Finish the below function for calculating a comprehensive discordance matrix.

In [ ]:
def getDiscordanceMatrix(data, criteria, parameters):
    discordance_matrix = np.zeros((len(data),len(data)))
    for A_idx, A_row in data.iterrows():
        for B_idx, B_row in data.iterrows():
            if A_idx != B_idx:
                #TODO
                continue
                
    return discordance_matrix

In [ ]:
getDiscordanceMatrix(data, criteria, parameters)

2.5) Now, finish the below function for generating the outranking matrix. This method should take into account both the concordance and discordance matrices.

In [ ]:
def getOutrankingMatrix(data, criteria, parameters, majority_treshold):
    concordance_matrix = getConcordanceMatrix(data, criteria, parameters, majority_treshold)
    discordance_matrix = getDiscordanceMatrix(data, criteria, parameters)
    n = len(data)
    outranking_matrix = np.zeros((n,n))
    ### TODO
    return outranking_matrix

In [ ]:
outranking_matrix = getOutrankingMatrix(data, criteria, parameters, majority_treshold=0.75)
print(outranking_matrix)

2.6) Change outranking matrix to adjacency list (dictionary) of a following form:

In [ ]:
def toAdjacencyList(outranking_matrix):
    n = len(outranking_matrix)
    graph = {i:[] for i in range(n)}
    ### TODO
    
    return graph

In [ ]:
graph = toAdjacencyList(outranking_matrix)
print(graph)

2.4) Draw outranking graph, and remove cycles (manually).

In [ ]:
cm.PrintGraph(graph, filename="graph_part_2")
plt.imshow(plt.imread("graph_part_2.png"))

2.5) **Question: Which mode of transport are in the kernel?**

# Part 3: Kernel

3.1) Given is the below outranking matrix

In [ ]:
outranking_matrix = np.array(   [[0., 1., 0., 0., 0., 0., 1., 0.],
                                 [0., 0., 1., 0., 1., 0., 0., 0.],
                                 [0., 0., 0., 0., 0., 0., 0., 1.],
                                 [0., 0., 0., 0., 1., 0., 1., 0.],
                                 [0., 0., 0., 0., 0., 1., 0., 0.],
                                 [0., 0., 0., 0., 0., 0., 1., 0.],
                                 [0., 0., 0., 0., 0., 0., 0., 1.],
                                 [0., 0., 0., 0., 0., 0., 0., 0.]]) 

In [ ]:
graph = toAdjacencyList(outranking_matrix)

In [ ]:
graph

3.2) Display the outranking graph. **Question: Which alternatives belong to kernel?**

In [ ]:
cm.PrintGraph(graph, filename="graph_part_3")
plt.imshow(plt.imread("graph_part_3.png"))

3.3) In this exercise, you are asked to complete the function for constructing a kernel. Firstly, complete the below auxiliary function which reverses edges of the graph (i.e., constructs reversed adjancency dictionary)

In [ ]:
def getReverseGraph(graph):
    n = len(graph)
    reverse_graph = {i:[] for i in graph}
    ### TODO
    
    return reverse_graph

3.4) Verify the correctness: compare the below reverse_graph with the above graph variable.

In [ ]:
reverse_graph = getReverseGraph(graph)
reverse_graph

3.2) Now, complete the below function for finding a graph kernel. This algorithm should proceed in the following way: <br>
1) Find non-outranked vertices. Add them to the kernel. <br> 
2) Remove the already found vertices from the graph and these vertices which are surpassed by them.<br>
3) Repeat (go to 1) until all vertices are removed from the graph. 

You can use the auxiliary reverse_graph structure. It can be helpfull for finding non-outranked vertices.

In [ ]:
def Kernel(graph):
    reverse_graph = getReverseGraph(graph)
    kernel = []
    step=0
    ## TODO 
    return kernel

In [ ]:
Kernel(graph)